In [4]:
import torch
import argparse
import numpy as np
import utils
import export

# Setup

parser = argparse.ArgumentParser()

parser.add_argument('--seed', type=int, default=123)
parser.add_argument('--method', type=str, default='SLL', choices=[
    'CER', 'REF', 'SLL', 'SLL_G'
])
parser.add_argument('--budget_pct', type=float, default=0.25)
parser.add_argument('--g0_method', type=str, default='random', choices=[
  'random', # randomly distribution of g0
  'large_cluster', # a random node and [g0_size] of its neighbors are in g0
  'many_clusters', # 10 random nodes and [g0_size] of their neighbors are in g0
  ])
parser.add_argument('--g0_size', type=float, default=0.2)
parser.add_argument('--lr', type=float, default=10)
parser.add_argument('--T_s', type=int, default=1263)
parser.add_argument('--T_u', type=int, default=-1)
parser.add_argument('--dataset', type=str, default='cora', choices=[
    'Cora', 'Cora-ML', 'Citeseer', 'Pubmed', 'Polblogs', 'ACM', 'BlogCatalog', 'Flickr', 'UAI'
])
parser.add_argument('--ptb_rate', type=float, default=0.25)
args = parser.parse_args("")

device = "cuda:1" if torch.cuda.is_available() else "cpu"
np.random.seed(args.seed)
torch.manual_seed(args.seed)

# Data


In [5]:
import random

def get_neighbor_subgraphs(adj: torch.Tensor, size: int, n: int) -> torch.Tensor:
    """
    Returns a tensor ~ `(n * size)` of node indices for `n` supgraphs of size `size`
    """
    res = torch.zeros([n, size], dtype=torch.long)

    for i in range(n):
        out = []
        stack = [random.randint(0, adj.shape[0] - 1)]
        while len(out) < size:
            if len(stack) == 0:
                stack.append(random.randint(0, adj.shape[0] - 1))
            curNode = stack.pop()
            if curNode not in out:
                out.append(curNode)
                children = adj[curNode].nonzero().t()[0].cpu().tolist()
                stack = children + stack
        res[i] = torch.tensor(out)

    return res

def get_rand_subgraphs(size: int, n: int) -> torch.Tensor:
    return torch.randint(adj.shape[0], [n, size], dtype=torch.long)


# Methods

In [6]:
from torch_geometric.nn import Sequential, DenseGCNConv
from torch.nn import Linear, ReLU
import torch.nn.functional as F
from tqdm import tqdm
from torch.utils.data import TensorDataset, DataLoader

def get_gcn(hid: int=64):
    return Sequential('x, adj', [
        (DenseGCNConv(feat.shape[1], hid), 'x, adj -> x'),
        ReLU(inplace=True),
        (DenseGCNConv(hid, hid), 'x, adj -> x'),
        ReLU(inplace=True),
        Linear(hid, int(label.max()) + 1),
    ]).to(device)

def train(model, dataloader: DataLoader, epochs: int):
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), 1e-3)
    t = tqdm(range(epochs), bar_format='{l_bar}{bar:10}{r_bar}{bar:-10b}')
    t.set_description("Model training")
    loss = torch.tensor(0)
    for _ in t:
        for feats, adjs, labels, train_masks in dataloader:
            pred = model(feats, adjs)
            mask = train_masks.flatten()
            loss = F.cross_entropy(pred.flatten(end_dim=1)[mask], labels.flatten(end_dim=1)[mask])
            loss.backward()
            optimizer.step()
            t.set_postfix({"loss": round(loss.item(), 2)})

def train_adj(model, feat, adj, label, train_mask, epochs: int):
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), 1e-3)
    t = tqdm(range(epochs), bar_format='{l_bar}{bar:10}{r_bar}{bar:-10b}')
    t.set_description("Model training")
    loss = torch.tensor(0)
    for _ in t:
        pred = model(feat, adj)
        loss = F.cross_entropy(pred.squeeze()[train_mask], label[train_mask])
        loss.backward()
        optimizer.step()
        t.set_postfix({"loss": round(loss.item(), 2)})

def eval(model, test_mask, text=""):
    model.eval()
    pred = model(feat, adj)
    acc = ((pred.argmax(dim=2).squeeze() == label)[test_mask].sum() / test_mask.sum()).item()
    print(text + f"Accuracy: {acc: 0.2%}")

In [7]:
num_samples = 256
sample_size = 32

In [8]:
adj, feat, label, train_mask, val_mask, test_mask = utils.load_data(args.dataset, args.seed, device=device)
pgd_subgraphs = get_neighbor_subgraphs(adj.cpu(), sample_size, num_samples)

Loading cora dataset...
Selecting 1 largest connected components


In [9]:
temp = adj[pgd_subgraphs]
subgraph_feats = feat[pgd_subgraphs]
subgraph_adjs = torch.zeros(num_samples, sample_size, sample_size).to(device)
subgraph_labels = label[pgd_subgraphs]
subgraph_train_masks = train_mask[pgd_subgraphs]
for i in range(num_samples):
    subgraph_adjs[i] = temp[i][:,pgd_subgraphs[i]]

In [10]:
clean_dataloader = DataLoader(
    TensorDataset(subgraph_feats, subgraph_adjs, subgraph_labels, subgraph_train_masks), 
    batch_size=128, shuffle=True)

gcn = get_gcn()
train(gcn, clean_dataloader, 100)
eval(gcn, test_mask)

Model training: 100%|██████████| 100/100 [00:03<00:00, 26.65it/s, loss=0.38]


Accuracy:  72.54%


In [16]:
import deeprobust
from deeprobust.graph.data import Dataset
from deeprobust.graph.defense import GCN
from deeprobust.graph.global_attack import PGDAttack
from deeprobust.graph.utils import preprocess

# Setup Victim Model
victim_model = GCN(nfeat=feat.shape[1], nclass=label.max().item()+1,
                    nhid=16, dropout=0.5, weight_decay=5e-4, device=device)
victim_model.fit(feat, adj, label, train_mask, verbose=True)

Epoch 0, training loss: 1.9161760807037354
Epoch 10, training loss: 1.0627546310424805
Epoch 20, training loss: 0.4512659013271332
Epoch 30, training loss: 0.2120225578546524
Epoch 40, training loss: 0.12651602923870087
Epoch 50, training loss: 0.09486710280179977
Epoch 60, training loss: 0.09469141066074371
Epoch 70, training loss: 0.0886521115899086
Epoch 80, training loss: 0.066765695810318
Epoch 90, training loss: 0.06861631572246552
Epoch 100, training loss: 0.0684695616364479
Epoch 110, training loss: 0.053196825087070465
Epoch 120, training loss: 0.06589846312999725
Epoch 130, training loss: 0.06179617717862129
Epoch 140, training loss: 0.05705156922340393
Epoch 150, training loss: 0.04810468852519989
Epoch 160, training loss: 0.04297434911131859
Epoch 170, training loss: 0.04362943395972252
Epoch 180, training loss: 0.044159047305583954
Epoch 190, training loss: 0.04222859442234039


In [18]:
# Setup Attack Model
model = PGDAttack(model=victim_model, nnodes=adj.shape[0], loss_type='CE', device=device)
model.attack(feat, adj, label, train_mask, n_perturbations=10)
modified_adj = model.modified_adj

100%|██████████| 200/200 [00:36<00:00,  5.44it/s]


In [ ]:
import torch_sparse

def normalize_adj_tensor(adj, sparse=False):
    """Normalize adjacency tensor matrix.
    """
    device = adj.device
    if sparse:
        # warnings.warn('If you find the training process is too slow, you can uncomment line 207 in deeprobust/graph/utils.py. Note that you need to install torch_sparse')
        # TODO if this is too slow, uncomment the following code,
        # but you need to install torch_scatter
        return normalize_sparse_tensor(adj)
        adj = to_scipy(adj)
        mx = normalize_adj(adj)
        return sparse_mx_to_torch_sparse_tensor(mx).to(device)
    else:
        mx = adj + torch.eye(adj.shape[0]).to(device)
        rowsum = mx.sum(1)
        r_inv = rowsum.pow(-1/2).flatten()
        r_inv[torch.isinf(r_inv)] = 0.
        r_mat_inv = torch.diag(r_inv)
        mx = r_mat_inv @ mx
        mx = mx @ r_mat_inv
    return mx

In [22]:
def get_modified_adj(complement, A, M):
    m = torch.zeros((A.shape[0], A.shape[0])).to(device)
    tril_indices = torch.tril_indices(row=A.shape[0], col=A.shape[0], offset=-1)
    m[tril_indices[0], tril_indices[1]] = M
    m = m + m.t()
    modified_adj = complement * m + A
    return modified_adj

In [23]:
import deeprutils
# ATTACK
A = adj
ε = 1000
X = feat
T = label
surrogate_lr = 1e-3
epochs = 30

M = torch.zeros_like(A).float().to(device)
θ = GCN(nfeat=X.shape[1], nclass=T.max().item()+1, nhid=32, lr=surrogate_lr, device=device).to(device)

t = tqdm(range(epochs), bar_format='{l_bar}{bar:10}{r_bar}{bar:-10b}')
A_p = torch.zeros_like(A).to(device).requires_grad_(True) # Initialize modified adj
complement = (torch.ones_like(adj) - torch.eye(adj.shape[0]).to(device) - A) - A

for epoch in t:
    A_p = get_modified_adj(complement, A, M)
    pred = θ(X, A_p)
    L = F.cross_entropy(pred, T)
    A_grad = torch.autograd.grad(L, A_p)[0]
    lr = 200 / np.sqrt(epoch + 1)
    M.data.add_(lr * A_grad)
    deeprutils.projection(ε, A_p)

    t.set_postfix({
        "loss": L.item(),
    })

  0%|          | 0/30 [00:00<?, ?it/s]


RuntimeError: shape mismatch: value tensor of shape [2485, 2485] cannot be broadcast to indexing result of shape [3086370]

In [20]:
import importlib
import deeprutils
importlib.reload(deeprutils)

<module 'deeprutils' from '/u/nyw6dh/HCDM/MismatchBatch/deeprutils.py'>

In [ ]:
import deeprutils

def attack(self, ori_features, adj, labels, idx_train, n_perturbations, epochs=200, **kwargs):
    complement = (torch.ones_like(adj) - torch.eye(adj.shape[0]).to(self.device) - adj) - adj
    def get_modified_adj(ori_adj, adj_changes):
        m = torch.zeros((adj.shape[0], adj.shape[0])).to(device)
        tril_indices = torch.tril_indices(row=adj.shape[0], col=adj.shape[0], offset=-1)
        m[tril_indices[0], tril_indices[1]] = adj_changes
        m = m + m.t()
        modified_adj = complement * m + ori_adj
        return modified_adj
    
    victim_model = get_gcn()
    victim_model.eval()
    adj_changes = torch.zeros_like(adj).to(device).requires_grad_(True)

    for t in tqdm(range(epochs)):
        modified_adj = get_modified_adj(adj, adj_changes)
        adj_norm = deeprutils.normalize_adj_tensor(modified_adj)
        output = victim_model(ori_features, adj_norm)
        # loss = F.nll_loss(output[idx_train], labels[idx_train])
        loss = F.cross_entropy(output[idx_train], labels[idx_train])
        adj_grad = torch.autograd.grad(loss, adj_changes)[0]

        lr = 200 / np.sqrt(t+1)
        adj_changes.data.add_(lr * adj_grad)

        deeprutils.projection(n_perturbations, adj_changes)

    self.random_sample(ori_adj, ori_features, labels, idx_train, n_perturbations)
    self.modified_adj = self.get_modified_adj(ori_adj).detach()
    self.check_adj_tensor(self.modified_adj)

In [ ]:
# def attack(model, dataloader: DataLoader, epochs: int):
surrogate = get_gcn()
surrogate.train()
optimizer = torch.optim.Adam(surrogate.parameters(), 1e-3)
t = tqdm(range(30), bar_format='{l_bar}{bar:10}{r_bar}{bar:-10b}')
t.set_description("Attacking training")
loss = torch.tensor(0)
for _ in t:
    for feats, adjs, labels, train_masks in clean_dataloader:
        pred = model(feats, adjs)
        mask = train_masks.flatten()
        loss = F.cross_entropy(pred.flatten(end_dim=1)[mask], labels.flatten(end_dim=1)[mask])
        loss.backward()
        optimizer.step()
        t.set_postfix({"loss": round(loss.item(), 2)})